In [ ]:
objname = "yasone2"

In [ ]:
import astroquery
import astropy.units as u
from astropy import coordinates

In [ ]:
radec = {
    "yasone2": [262.348711541485, 6.38646541032259]
}[objname]

In [ ]:
coord0 = coordinates.SkyCoord(radec[0], radec[1], unit=u.deg)

In [ ]:
from astroquery.gaia import Gaia

In [ ]:
Gaia.ROW_LIMIT = 100_000
gaia_sources = Gaia.query_object(coordinate=coord0, radius=30*u.arcmin, verbose=True)

In [ ]:
len(gaia_sources)

In [ ]:
assert len(gaia_sources) < Gaia.ROW_LIMIT

In [ ]:
gaia_sources

In [ ]:
gaia_sources.write(f"{objname}_Gaia.csv")